In [1]:
import psycopg2
from psycopg2.extras import execute_values
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
connection = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="63333", database="stack_overflow")

In [4]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select title, body
        from posts_19
        limit 5""")
        posts = cursor.fetchall()

In [5]:
posts[:2]

[('Accessing Twitter Data: API/Cursor (twitter user) vs Streaming (hash_tag_list)',
  "<p>I was studying the code to access Twitter data. The code was written by someone whos shows how to access the twitter data in youtube.</p>\n<p>Please see code below (some parts are truncated):</p>\n<pre><code>from tweepy import API \nfrom tweepy import Cursor\nfrom tweepy.streaming import StreamListener\nfrom tweepy import OAuthHandler\nfrom tweepy import Stream\n \nimport twitter_credentials\nimport numpy as np\nimport pandas as pd\n\nclass TwitterClient():\n    def __init__(self, twitter_user=None):\n        self.auth = TwitterAuthenticator().authenticate_twitter_app()\n        self.twitter_client = API(self.auth)\n        self.twitter_user = twitter_user\n    def get_user_timeline_tweets(self, num_tweets):\n        tweets = []\n        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):\n            tweets.append(tweet)\n        return tweets        \n

In [6]:
soup = BeautifulSoup(posts[0][1])
print(soup.get_text())

I was studying the code to access Twitter data. The code was written by someone whos shows how to access the twitter data in youtube.
Please see code below (some parts are truncated):
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
 
import twitter_credentials
import numpy as np
import pandas as pd

class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets        

class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = xxxx
        return auth

class TwitterStreamer(

# Extract and transform to text all posts

In [7]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""SELECT count(*)
            FROM information_schema.columns
            WHERE table_name = 'posts_19'
            AND column_name = 'body_text'
        """)
        if cursor.fetchall()[0][0] == 0:
            cursor.execute("ALTER TABLE posts_19 ADD COLUMN body_text TEXT;")
            print("Column created")
        else:
            print("Column already exists")

Column already exists


In [8]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select count(*)
        from posts_19 where body is not null""")
        total_posts = cursor.fetchall()[0][0]
total_posts

17069451

In [9]:
def extract_text(id_body):
    post_id, body = id_body
    return post_id, BeautifulSoup(body, 'html.parser').get_text().encode("utf-8", 'backslashreplace').decode().replace('\0', '\\0')

In [ ]:
batch_size = 300000
total_batches = (total_posts//batch_size)+1

for batch_index in range(total_batches):

    with connection:
        with connection.cursor() as cursor:
            cursor.execute(f"""select id, body
            from posts_19
            where body is not null
            order by id
            limit {batch_size} offset {batch_size*batch_index}""")
            posts = cursor.fetchall()
            
    with Pool(8) as p:
        maped_posts = p.map(extract_text, posts)
    
    with connection:
        with connection.cursor() as cursor:
            execute_values(cursor, """UPDATE posts_19 SET body_text = data.body_text
                                    FROM (VALUES %s) AS data (id, body_text)
                                    WHERE posts_19.id = data.id""",
                                    maped_posts)
    print(f"{(batch_index+1)/total_batches*100:.2f}% posts procesados")

In [23]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select count(*) from posts_19 where body_text is not null;")
        posts_body_sample = cursor.fetchall()[0][0]
posts_body_sample

17069451

In [24]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select id, body, body_text from posts_19 where body_text is not null limit 3;")
        posts_body_sample = cursor.fetchall()
posts_body_sample

[(68451754,
  '<p>We are currently developing an AppEngine Standard application using Cloud Firestore in Native Mode which is working very well.\nNow, we want to support a query that needs a composite index which we try to create using a <code>index.yaml</code> file:</p>\n<pre><code>indexes:\n  - kind: myentity\n    properties:\n      - name: owner\n        direction: asc\n      - name: created_at\n        direction: asc\n</code></pre>\n<p>When testing this with <code>gcloud app deploy index.yaml</code>, we observe that indexes are being created. However, the created indexes are Cloud Datastore indexes not Cloud Firestore ones, even though Datastore is &quot;not even enabled&quot; (The Cloud console wants to send you directly to Firestore view if you try to access the Datastore view).\nTherefore, our queries are failing.\nHow do we instruct AppEngine to create Firestore indexes not Datastore indexes?</p>\n',
  'We are currently developing an AppEngine Standard application using Cloud F